In [39]:
# ZILLOW DATA EXTRACTION WRITTEN BY SONIA YANG

# Dependencies
import requests
import urllib
import random
import math
import pandas as pd
import xml.etree.ElementTree as ET
import time
from config import zws_id, gkey # please use your own Zillow & Google API keys!
from urllib.request import urlopen

In [40]:
# FUNCTION to grab the exact address based on longitude and latitude
# modified from here https://gist.github.com/bradmontgomery/5397472
# their example didn't include an API key, but I added it otherwise you'd hit the rate limit easily

def reverse_geocode(latitude, longitude):
    # Did the geocoding request comes from a device with a
    # location sensor? Must be either true or false
    sensor = 'true'

    # Hit Google's reverse geocoder directly
    # NOTE: I *think* their terms state that you're supposed to
    # use google maps if you use their api for anything.
    base = "https://maps.googleapis.com/maps/api/geocode/json?"
    params = "latlng={lat},{lon}&sensor={sen}&key={key}".format(
        lat=latitude,
        lon=longitude,
        sen=sensor,
        key=gkey
    )
    url = "{base}{params}".format(base=base, params=params)
    #print(url)
    response = requests.get(url).json()
    address = response['results'][0]['formatted_address']
    return address


In [41]:
# FUNCTION to generate random lat & lng within a certain radius 
# modified from here: http://hadoopguru.blogspot.com/2014/12/python-generate-random-latitude-and.html
# changed to take in an empty initial dataframe and load in the data + return it
# this calls the reverse geocode function to grab the addresses of each randomly generated lat & lng

def generate_addresses(latitude, longitude, df):
    
    radius = 10000                         #Choose your own radius
    radiusInDegrees=radius/111300            
    r = radiusInDegrees

    counter = 0
    
    for i in range(1,200):                 #Choose number of Lat Long to be generated

        u = float(random.uniform(0.0,1.0))
        v = float(random.uniform(0.0,1.0))

        w = r * math.sqrt(u)
        t = 2 * math.pi * v
        x = w * math.cos(t) 
        y = w * math.sin(t)

        xLat  = x + latitude
        yLng = y + longitude

        df.set_value(counter, "latitude", xLat)
        df.set_value(counter, "longitude", yLng)
        
        #print(format(counter) + ": " + format(xLat) + ", " + format(yLng))
        address = reverse_geocode(xLat, yLng).split(',')
        citystatezip = address[1] + address[2]
        
        df.set_value(counter, "address", address[0])
        df.set_value(counter, "city_state_zip", citystatezip)
        
        # Add to counter
        counter = counter + 1
    
    return df

In [42]:
# FUNCTION to call Zillow API's GetSearchResults and will check to see if a house exists at that address
# message code will be written to dataframe
# zillow url format
# http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=<ZWSID>&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA
        
def get_message_codes(df):

    for index, row in df.iterrows():

        try:
            url = 'https://www.zillow.com/webservice/GetSearchResults.htm?zws-id='
            address = row['address']
            citystatezip =row['city_state_zip']


            query_url = url + zws_id + '&address=' + urllib.parse.quote(address) + '&citystatezip=' + urllib.parse.quote(citystatezip) 
            #print(query_url)

            root = ET.parse(urlopen(query_url)).getroot()

            for message in root.iter('message'):
                message_code = message[1].text

            print(format(index) + ": " + message_code)

            df.set_value(index, 'message_code', message_code)

            time.sleep(0.5) #necessary bc bombarding Zillow with API calls doesn't allow enough time to respond to each

        except:
            break
    

In [43]:
# FUNCTION to call Zillow's GetDeepSearchResults and look up Zestimate, bed, and bath
# http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=<ZWSID>&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA
# there are some limitations such as multiple zestimates depending on when the house was sold/if it was sold multiple times
# the code to handle that would get too convoluted so I am just writing in the most recent (according to the API) values
# probably not what we would do in real life
# but a decision we made re: the scope of a classroom project on a short time constraint

def search_zillow(df):
    
    for index, row in df.iterrows():
        try:
            url = 'https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id='
            address = df['address'][index]
            citystatezip = df['city_state_zip'][index]


            query_url = url + zws_id + '&address=' + urllib.parse.quote(address) + '&citystatezip=' + urllib.parse.quote(citystatezip) 


            root = ET.parse(urlopen(query_url)).getroot()

            print("row " + format(index) + ": " + address + citystatezip)
            print(query_url)

            '''
               "year built","lot size","finished sq ft"'''
            
            #zpid
            for zpid in root.iter('zpid'):
                df.set_value(index,'zpid', zpid.text)
            
            # we already have the address from the address + citystatezip variables
            # so we don't need to grab it again
            # same with lat & lng already being in the table
            
            #valuation (high and low)
            for valuation in root.iter('valuationRange'):
                highValuation = valuation[1].text
                lowValuation = valuation[0].text
                df.set_value(index, 'valuation_high', highValuation)
                df.set_value(index, 'valuation_low', lowValuation)
            
            #zestimate
            for zestimate in root.iter('zestimate'):
                zestimate_value = zestimate[0].text

                if zestimate_value is None:
                    print('not for sale')
                else:
                    print ('zestimate (value): ' + format(zestimate[0].text)) 
                    df.set_value(index, 'zestimate', zestimate_value)
             
            #home value index
            for zindexValue in root.iter('zindexValue'):
                df.set_value(index, 'home value index', zindexValue.text)
            
            #tax assessment
            for taxAssessment in root.iter('taxAssessment'):
                df.set_value(index, 'tax assessment', taxAssessment.text)
                
            #tax assessment year
            for taxAssessmentYear in root.iter('taxAssessmentYear'):
                df.set_value(index, 'tax assess year', taxAssessmentYear.text)
                
            #year built
            for yearBuilt in root.iter('yearBuilt'):
                df.set_value(index, 'year built', yearBuilt.text)
             
            #lot size sq ft
            for lotSizeSqFt in root.iter('lotSizeSqFt'):
                df.set_value(index, 'lot size', lotSizeSqFt.text)
            
            #finished sq ft
            for finishedSqFt in root.iter('finishedSqFt'):
                df.set_value(index, 'finished sq ft', finishedSqFt.text)
            
            #bedrooms
            for bedroom in root.iter('bedrooms'):
                bedrooms = bedroom.text
                #print("bedrooms: " + bedrooms)
                df.set_value(index, 'bedrooms', bedrooms)

            #bathrooms
            for bathroom in root.iter('bathrooms'):
                bathrooms = bathroom.text
                #print("bathrooms: " + bathrooms + "\n")
                df.set_value(index, 'bathrooms', bathrooms)           
            
            print('\n')

            time.sleep(0.5) 


        except:
            break


<h2>HOW TO RUN THIS CODE</h2>
<ol>
<li>Initialize an empty dataframe with the fields as marked below</li>
<li>Call the <strong>generate_addresses</strong> function passing in your empty dataframe</li>
<li>Call the <strong>get_message_codes</strong> to update your dataframe with message codes indicating whether or not a valid property exists at each address. <strong>IMPORTANT:</strong> please register your own Zillow account/get your own key for this!! If we all keep using the same one we'll easily hit the rate limit </li>
<li>Drop the rows in the dataframe for which a property does not exist at that address</li>
<li>Call the <strong>search_zillow</strong> function to get the zestimate (aka price of the property), # of bedrooms, and # of bathrooms</li>
<li>I did not include it in my code, but once you get a sample size of data that you are satisfied with for the city, maybe write it out to a CSV so you don't have to keep running this code/can use it later</li>
</ol>

feel free to comment out my print statements while the functions are running if you find them distracting

In [44]:
# HOW TO RUN ALL THE FUNCTIONS, USING LOS ANGELES AS AN EXAMPLE

# coordinates taken from the CitiesGeo_Output.csv
# we should manually run the following code on each individual city instead of nesting it in another loop
# while this may be hardcoded, it's better than waiting on one gigantic loop that takes forever

# STEP 1: INITALIZE THE DATAFRAME
# if we need any more fields, let me know
la_df = pd.DataFrame({"zpid": '',
                      "address":'',
                      "city_state_zip":'',
                      "latitude":'',
                      "longitude":'',
                      "message_code":'',
                      "zestimate":'',
                      "valuation_high":'',
                      "valuation_low": '',
                      "home value index":'',
                      "tax assessment":'',
                      "tax assess year":'',
                      "year built":'',
                      "lot size":'',
                      "finished sq ft":'',
                      "bedrooms":'',
                      "bathrooms":''}, index=[0])

# reorder the columns
la_df = la_df[["zpid", "address","city_state_zip","latitude","longitude","message_code","zestimate",
               "valuation_high","valuation_low","home value index","tax assessment","tax assess year",
               "year built","lot size","finished sq ft","bedrooms","bathrooms"]]

# STEP 2: GENERATE RANDOM ADDRESSES IN THE DESIGNATED AREA
# pass in the coordinates for Los Angeles plus the empty dataframe
generate_addresses(34.0522342,-118.2436849, la_df) 

#la_df

,zpid,address,city_state_zip,latitude,longitude,message_code,zestimate,valuation_high,valuation_low,home value index,tax assessment,tax assess year,year built,lot size,finished sq ft,bedrooms,bathrooms
0,,4651 Castle Crest Dr,Los Angeles CA 90041,34.1344,-118.223,,,,,,,,,,,,
1,NaN,3933 Ingraham St,Los Angeles CA 90005,34.0607,-118.312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3221 Fruitland Ave,Los Angeles CA 90058,33.9972,-118.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4443 Verdugo Rd,Los Angeles CA 90065,34.132,-118.233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1241 S Grand Ave,Los Angeles CA 90015,34.0395,-118.264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,520 S Lucerne Blvd,Los Angeles CA 90020,34.0646,-118.324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,638 Pheasant Dr,Los Angeles CA 90065,34.1042,-118.212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,107 S Vermont Ave,Los Angeles CA 90004,34.0724,-118.292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1847-1899 S San Pedro St,Los Angeles CA 90015,34.028,-118.257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2301 S Hoover St,Los Angeles CA 90007,34.0349,-118.284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# STEP 3: CALL THE ZILLOW API TO GET MESSAGE CODES
# 0 means there is a valid property at that address
# 508 and anything else means there isn't
# if you get nothing but invalid message codes, re-run STEP 2
# you might have to sign up for a new Zillow account if you keep getting invalid results here
# there is a possibility you hit the rate limit

get_message_codes(la_df)

0: 0
1: 508
2: 508
3: 0
4: 508
5: 0
6: 0
7: 508
8: 508
9: 0
10: 508
11: 0
12: 0
13: 0
14: 508
15: 0
16: 508
17: 508
18: 508
19: 0
20: 508
21: 508
22: 0
23: 508
24: 508
25: 508
26: 508
27: 508
28: 508
29: 508
30: 508
31: 508
32: 0
33: 0
34: 508
35: 508
36: 508
37: 0
38: 508
39: 508
40: 0
41: 508
42: 0
43: 0
44: 508
45: 508
46: 0
47: 0
48: 0
49: 508
50: 508
51: 508
52: 0
53: 0
54: 508
55: 508
56: 508
57: 508
58: 508
59: 508
60: 508
61: 508
62: 508
63: 508
64: 0
65: 508
66: 508
67: 0
68: 508
69: 0
70: 508
71: 508
72: 508
73: 508
74: 0
75: 508
76: 508
77: 0
78: 0
79: 0
80: 0
81: 508
82: 508
83: 508
84: 508
85: 508
86: 0
87: 508
88: 0
89: 508
90: 508
91: 508
92: 508
93: 508
94: 508
95: 0
96: 508
97: 508
98: 0
99: 508
100: 508
101: 508
102: 508
103: 0
104: 0
105: 0
106: 508
107: 508
108: 0
109: 508
110: 0
111: 508
112: 508
113: 508
114: 508
115: 508
116: 0
117: 508
118: 508
119: 508
120: 0
121: 508
122: 0
123: 508
124: 508
125: 0
126: 508
127: 508
128: 0
129: 0
130: 508
131: 508
132: 0
133: 

In [46]:
# STEP 4: DROP INVALID ENTRIES FROM DATAFRAME 
# cull all the rows where houses do not exist at the address
# take what is valid (message code of '0')
# the code sometimes might break/not get a response from the server so it's better to take what IS valid

la_df = la_df[la_df.message_code == '0']
la_df

,zpid,address,city_state_zip,latitude,longitude,message_code,zestimate,valuation_high,valuation_low,home value index,tax assessment,tax assess year,year built,lot size,finished sq ft,bedrooms,bathrooms
0,,4651 Castle Crest Dr,Los Angeles CA 90041,34.1344,-118.223,0,,,,,,,,,,,
3,NaN,4443 Verdugo Rd,Los Angeles CA 90065,34.132,-118.233,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,520 S Lucerne Blvd,Los Angeles CA 90020,34.0646,-118.324,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,638 Pheasant Dr,Los Angeles CA 90065,34.1042,-118.212,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2301 S Hoover St,Los Angeles CA 90007,34.0349,-118.284,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,2880 E Gage Ave,Huntington Park CA 90255,33.9804,-118.218,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,1305 Romulus Dr,Glendale CA 91205,34.1347,-118.237,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,4772 Academy St,Los Angeles CA 90032,34.0864,-118.182,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,3529 La Clede Ave,Los Angeles CA 90039,34.1198,-118.257,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,3226 Lowell Ave,Los Angeles CA 90032,34.0818,-118.162,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# STEP 5: SEARCH ZILLOW AND GET ZESTIMATE, BEDROOMS, & BATHROOMS
# fill the dataframe with the data

search_zillow(la_df)
la_df

row 0: 4651 Castle Crest Dr Los Angeles CA 90041
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=4651%20Castle%20Crest%20Dr&citystatezip=%20Los%20Angeles%20CA%2090041
zestimate (value): 720784


row 3: 4443 Verdugo Rd Los Angeles CA 90065
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=4443%20Verdugo%20Rd&citystatezip=%20Los%20Angeles%20CA%2090065
zestimate (value): 835577


row 5: 520 S Lucerne Blvd Los Angeles CA 90020
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=520%20S%20Lucerne%20Blvd&citystatezip=%20Los%20Angeles%20CA%2090020
zestimate (value): 3521935


row 6: 638 Pheasant Dr Los Angeles CA 90065
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=638%20Pheasant%20Dr&citystatezip=%20Los%20Angeles%20CA%2090065
zestimate (value): 890593


row 9: 2301 S Hoover St Los Angeles CA 90007
ht

row 98: 331 S Ave 57 Los Angeles CA 90042
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=331%20S%20Ave%2057&citystatezip=%20Los%20Angeles%20CA%2090042
zestimate (value): 1021369
zestimate (value): 815420


row 103: 1706 W Jefferson Blvd Los Angeles CA 90018
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=1706%20W%20Jefferson%20Blvd&citystatezip=%20Los%20Angeles%20CA%2090018
zestimate (value): 796701


row 104: 152 S Lucerne Blvd Los Angeles CA 90004
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=152%20S%20Lucerne%20Blvd&citystatezip=%20Los%20Angeles%20CA%2090004
zestimate (value): 1605474


row 105: 3400 Carnation Ave Los Angeles CA 90026
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=3400%20Carnation%20Ave&citystatezip=%20Los%20Angeles%20CA%2090026
zestimate (value): 1705076


row 108:

row 193: 4218 S Hobart Blvd Los Angeles CA 90062
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=4218%20S%20Hobart%20Blvd&citystatezip=%20Los%20Angeles%20CA%2090062
zestimate (value): 496420


row 194: 5418 Meridian St Los Angeles CA 90042
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=5418%20Meridian%20St&citystatezip=%20Los%20Angeles%20CA%2090042
zestimate (value): 677017


row 195: 705 Milwaukee Ave Los Angeles CA 90042
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=705%20Milwaukee%20Ave&citystatezip=%20Los%20Angeles%20CA%2090042
zestimate (value): 744632




,zpid,address,city_state_zip,latitude,longitude,message_code,zestimate,valuation_high,valuation_low,home value index,tax assessment,tax assess year,year built,lot size,finished sq ft,bedrooms,bathrooms
0,20850860,4651 Castle Crest Dr,Los Angeles CA 90041,34.1344,-118.223,0,720784,764031,684745,"743,500",850000.0,2017,1949,9256,1429,2,2.0
3,20848463,4443 Verdugo Rd,Los Angeles CA 90065,34.132,-118.233,0,835577,894067,777087,"691,000",51908.0,2017,1951,6282,1563,3,2.0
5,20775277,520 S Lucerne Blvd,Los Angeles CA 90020,34.0646,-118.324,0,3521935,3768470,3310619,"1,542,600",2683422.0,2017,1920,13372,3692,4,4.0
6,20760983,638 Pheasant Dr,Los Angeles CA 90065,34.1042,-118.212,0,890593,952935,846063,"709,000",390661.0,2017,1941,8879,1440,3,2.0
9,2111194430,2301 S Hoover St,Los Angeles CA 90007,34.0349,-118.284,0,789439,1105215,584185,NaN,NaN,NaN,1968,NaN,NaN,0,NaN
11,62018650,2880 E Gage Ave,Huntington Park CA 90255,33.9804,-118.218,0,477877,501771,453983,"363,100",42105.0,2017,1948,4004,3694,5,5.0
12,20848238,1305 Romulus Dr,Glendale CA 91205,34.1347,-118.237,0,993876,1053509,904427,"510,200",636051.0,2017,1948,7886,2484,3,2.0
13,20639008,4772 Academy St,Los Angeles CA 90032,34.0864,-118.182,0,531241,557803,483429,"446,600",104483.0,2017,1910,5000,912,2,1.0
15,20750599,3529 La Clede Ave,Los Angeles CA 90039,34.1198,-118.257,0,809011,849462,760470,"764,600",238955.0,2017,1921,6753,1344,3,2.0
19,125242705,3226 Lowell Ave,Los Angeles CA 90032,34.0818,-118.162,0,776066,814869,737263,"446,600",NaN,2017,NaN,5996,NaN,NaN,NaN


In [48]:
# do any further data cleaning you need to yourself
# for example, dropping any rows with NaN values
la_df = la_df.dropna(axis=0, how='any')
la_df

# maybe write to CSV to store the data for usage later/before doing plots? so you don't have to rerun everything

,zpid,address,city_state_zip,latitude,longitude,message_code,zestimate,valuation_high,valuation_low,home value index,tax assessment,tax assess year,year built,lot size,finished sq ft,bedrooms,bathrooms
0,20850860,4651 Castle Crest Dr,Los Angeles CA 90041,34.1344,-118.223,0,720784,764031,684745,"743,500",850000.0,2017,1949,9256,1429,2,2.0
3,20848463,4443 Verdugo Rd,Los Angeles CA 90065,34.132,-118.233,0,835577,894067,777087,"691,000",51908.0,2017,1951,6282,1563,3,2.0
5,20775277,520 S Lucerne Blvd,Los Angeles CA 90020,34.0646,-118.324,0,3521935,3768470,3310619,"1,542,600",2683422.0,2017,1920,13372,3692,4,4.0
6,20760983,638 Pheasant Dr,Los Angeles CA 90065,34.1042,-118.212,0,890593,952935,846063,"709,000",390661.0,2017,1941,8879,1440,3,2.0
11,62018650,2880 E Gage Ave,Huntington Park CA 90255,33.9804,-118.218,0,477877,501771,453983,"363,100",42105.0,2017,1948,4004,3694,5,5.0
12,20848238,1305 Romulus Dr,Glendale CA 91205,34.1347,-118.237,0,993876,1053509,904427,"510,200",636051.0,2017,1948,7886,2484,3,2.0
13,20639008,4772 Academy St,Los Angeles CA 90032,34.0864,-118.182,0,531241,557803,483429,"446,600",104483.0,2017,1910,5000,912,2,1.0
15,20750599,3529 La Clede Ave,Los Angeles CA 90039,34.1198,-118.257,0,809011,849462,760470,"764,600",238955.0,2017,1921,6753,1344,3,2.0
22,21030425,3160 California St,Huntington Park CA 90255,33.9708,-118.211,0,423525,444701,402349,"363,100",325320.0,2017,1926,6011,996,2,1.0
32,20635402,1118 Calzona St,Los Angeles CA 90023,34.0203,-118.195,0,374335,393052,344388,"379,500",111501.0,2017,1922,6870,756,2,1.0
